In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
all_beauty_path = 'data/All_Beauty.csv'
# all_beauty_path = 'http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv'
ratings = pd.read_csv(
    all_beauty_path,
    names=["asin", "reviewerID", "overall", "unixReviewTime"],
    dtype={"asin": np.str_, "reviewerID": np.str_, "overall": np.float32, "unixReviewTime": np.float64 }
)
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
ratings.head()

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1.424304e+09,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1.418861e+09,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1.407629e+09,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1.362960e+09,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1.324771e+09,2011-12-25


In [16]:
meta_all_beauty_path = 'data/meta_All_Beauty.json.gz'
# meta_all_beauty_path = 'http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz'
metadata = pd.read_json(
    meta_all_beauty_path,
    lines=True,
    compression="gzip"
)
metadata.head()
print(metadata.columns)

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'details', 'main_cat',
       'similar_item', 'date', 'price', 'asin', 'imageURL', 'imageURLHighRes'],
      dtype='object')


# 資料切分

In [17]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') &
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())


In [14]:
d = next(ratings_trainings.iterrows())[1]
d.iloc('reviewerID')

ValueError: No axis named reviewerID for object type Series

In [22]:
d = {
    'a': {'i': 1},
    'b': 2
}

In [24]:
d.items()

dict_items([('a', {'i': 1}), ('b', 2)])

In [25]:
init_sim = lambda x  : [0 + x for _ in range(3)]
init_sim(1)


[1, 1, 1]

In [27]:
factory = lambda: defaultdict(init_sim)

In [28]:
factory()


NameError: name 'defaultdict' is not defined

In [9]:
df = pd.DataFrame([[1, 2], [4, 5], [7, 8]],
     index=['cobra', 'viper', 'sidewinder'],
     columns=['max_speed', 'shield'])
df

,max_speed,shield
cobra,1,2
viper,4,5
sidewinder,7,8


In [14]:
df.loc['viper']

max_speed    4
shield       5
Name: viper, dtype: int64

In [11]:
df['max_speed'] == 1


cobra          True
viper         False
sidewinder    False
Name: max_speed, dtype: bool

In [23]:
ratings_trainings

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1.424304e+09,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1.418861e+09,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1.407629e+09,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1.362960e+09,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1.324771e+09,2011-12-25
...,...,...,...,...,...
371340,B01HJEGTYK,A202DCI7TV1022,1.0,1.500509e+09,2017-07-20
371341,B01HJEGTYK,A3FSOR5IJOFIBE,5.0,1.489622e+09,2017-03-16
371342,B01HJEGTYK,A1B5DK6CTP2P24,5.0,1.488326e+09,2017-03-01
371343,B01HJEGTYK,A23OUYS5IRMJS9,2.0,1.487635e+09,2017-02-21


In [24]:
# ratings_trainings['reviewerID'] == 'A1V6B6TNIC10QE'
ratings_trainings.loc[ratings_trainings['reviewerID'] == 'A1V6B6TNIC10QE' ]

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1.424304e+09,2015-02-19


In [22]:
ratings_trainings.loc[ratings_trainings['reviewerID'] == 'A1V6B6TNIC10QE' ]

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1.424304e+09,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1.418861e+09,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1.407629e+09,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1.362960e+09,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1.324771e+09,2011-12-25
...,...,...,...,...,...
371340,B01HJEGTYK,A202DCI7TV1022,1.0,1.500509e+09,2017-07-20
371341,B01HJEGTYK,A3FSOR5IJOFIBE,5.0,1.489622e+09,2017-03-16
371342,B01HJEGTYK,A1B5DK6CTP2P24,5.0,1.488326e+09,2017-03-01
371343,B01HJEGTYK,A23OUYS5IRMJS9,2.0,1.487635e+09,2017-02-21
